# Getting stuff ready

In [ ]:
gh repo clone ruizheliUOA/TWR-VAE

Cloning into 'BigBiGAN-PyTorch'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 57 (delta 18), reused 51 (delta 15), pack-reused 0
Unpacking objects: 100% (57/57), done.


In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install pytorch-fid

  Created wheel for pytorch-fid: filename=pytorch_fid-0.2.0-cp37-none-any.whl size=10547 sha256=c2825fcd1973db8cb30dc5ea576a27283cfb74bb023e5f02b2b7589e12854eb7
  Stored in directory: /root/.cache/pip/wheels/72/36/3c/4f3fb256f62d24bef52636f66f21667bc21caa637ce92f0e53
Successfully built pytorch-fid


# Changing loss modes: 

in the following codes, different loss modes are used for model training:

major changes in the code are in train_gan.py, pipeline.py and model/losses.py. Whereever there is a change there is a comment section like this: 

\# Loss modes: description of change ###################

my updated code

\##############################################

## Using all loss parts:

some arguments are added to train_gan.py as follows: 

--loss_mode -> choices: ("all", "no_sx", "no_sz", "no_sxz", "no_sx_sz", "info_gan")

-- train -> choices: (0, 1) indicates whether we are training or testing

-- resume -> when in training mode we can load from checkpoint and specify which epoch to resume training from

#### Train

as we see model is trained for 31 epochs and the last acheived loss is this:

epoch 31, disc_loss 1.330464482307434, gen_enc_loss 1.0490446090698242

In [ ]:
!python "/content/drive/MyDrive/BigBiGAN-PyTorch-main/train_gan.py" --dataset "MNIST" --data_path "/content/drive/MyDrive/BigBiGAN-PyTorch-main/data/" --train 1 --loss_mode "all" --resume 22

Loading data from checkpoint, resuming from epoch: 22
creating model from checkpoint
2021-05-18 17:00:15.789994: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
0it [00:00, ?it/s]epoch 22, disc_loss 2.100839614868164, gen_enc_loss 0.8357255458831787
50it [01:02,  1.23s/it]epoch 22, disc_loss 1.4718999862670898, gen_enc_loss 0.8557757139205933
100it [02:04,  1.24s/it]epoch 22, disc_loss 1.5015583038330078, gen_enc_loss 0.9620881080627441
150it [03:08,  1.27s/it]epoch 22, disc_loss 1.444177508354187, gen_enc_loss 0.9139944314956665
200it [04:10,  1.25s/it]epoch 22, disc_loss 1.5130914449691772, gen_enc_loss 0.9368475675582886
234it [04:53,  1.26s/it]
0it [00:00, ?it/s]epoch 23, disc_loss 1.4531590938568115, gen_enc_loss 0.882318913936615
50it [01:04,  1.27s/it]epoch 23, disc_loss 1.4208626747131348, gen_enc_loss 0.9608749151229858
100it [02:07,  1.27s/it]epoch 23, disc_loss 1.5061206817626953, gen_enc_loss 0.9402112960

### Test

in this mode a batch of original images are saved in "data/org/" directory and a batch of generated images are saved in "data/MNIST/{loss_mode}/gen/"  for later FID calculation and InceptionScore is calculated in the process. Furthermore encoded train and test codes (encoded images) with their labels are saved as CSV in "data/MNIST/{loss_mode}/encoded/"

as we see InceptionScore in this mode is  1.1652391

In [ ]:
!python "/content/drive/MyDrive/BigBiGAN-PyTorch-main/train_gan.py" --dataset "MNIST" --data_path "/content/drive/MyDrive/BigBiGAN-PyTorch-main/data/" --train 0 --loss_mode "all"

original images saved
creating generated image directory (save one batch
0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
/content/drive/MyDrive/BigBiGAN-PyTorch-main/src/pipeline/pipeline.py:235: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x).data.cpu().numpy()
___________________________________
Inception Score is ::::::  1.1652391
___________________________________
done
0it [00:04, ?it/s]
generated images saved
encoded test csv saved
encoded train csv saved


### FID

pytorch FID library is used. because our images are very small compared to original input images af InceptionNet, --dims is chosen as 192 which implies using a shallower CNN structure

FID in this mode is 0.3373776426348982

In [ ]:
!python -m pytorch_fid "/content/drive/MyDrive/BigBiGAN-PyTorch-main/data/org/" "/content/drive/MyDrive/BigBiGAN-PyTorch-main/data/MNIST/bigbigan/all/gen/" --dims 192 --device "cuda"

Downloading: "https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/pt_inception-2015-12-05-6726825d.pth
100% 91.2M/91.2M [00:01<00:00, 56.0MB/s]
100% 6/6 [01:08<00:00, 11.41s/it]
100% 6/6 [00:02<00:00,  2.97it/s]
FID:  0.3373776426348982


### Linear Accuracy

KNN (n = 3) classifier is used and accuracy on test is 0.15775240384615385 when trained for 30 epochs. When I trained on only 10 epochs, it would only yield 10% accuracy. I believe more accuracy can be acheived with more training as the decoder learns to succesfully decode and generate images. Furthermore in VAE because the encoder tries to learn a latent with little to no covvariance among it's dimensions, it is plausible to have low linear accuracy

linear accuracy is 0.15775240384615385

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/BigBiGAN-PyTorch-main/data/MNIST/bigbigan/all/encoded/train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/BigBiGAN-PyTorch-main/data/MNIST/bigbigan/all/encoded/test.csv")

train_y, train_X = train_df.loc[:, "0"], train_df.loc[:, "1":"100"]
test_y, test_X = test_df.loc[:, "0"], test_df.loc[:, "1":"100"]

neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(train_X, train_y)
neigh.score(test_X, test_y)

0.15775240384615385

## No SZ

### Training

--loss_mode: "no_sz"

training is done for 9 epochs and loss is as follows:

epoch 9, disc_loss 1.576092004776001, gen_enc_loss 1.3424787521362305

In [ ]:
!python "/content/drive/MyDrive/BigBiGAN-PyTorch-main/train_gan.py" --dataset "MNIST" --data_path "/content/drive/MyDrive/BigBiGAN-PyTorch-main/data/" --train 1 --loss_mode "no_sz"

no_sz
2021-05-13 08:15:59.441843: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
0it [00:00, ?it/s]epoch 0, disc_loss 3.277395248413086, gen_enc_loss 1.4751172065734863
50it [00:47,  1.05it/s]epoch 0, disc_loss 0.038688644766807556, gen_enc_loss 5.039902687072754
100it [01:35,  1.04it/s]epoch 0, disc_loss 0.08507563173770905, gen_enc_loss 4.818063735961914
150it [02:24,  1.03it/s]epoch 0, disc_loss 0.07505977153778076, gen_enc_loss 4.643559455871582
200it [03:12,  1.02it/s]epoch 0, disc_loss 0.24523942172527313, gen_enc_loss 4.300935745239258
234it [03:46,  1.03it/s]
0it [00:00, ?it/s]epoch 1, disc_loss 0.21476471424102783, gen_enc_loss 3.9550118446350098
50it [00:55,  1.10s/it]epoch 1, disc_loss 0.14024361968040466, gen_enc_loss 3.992415428161621
100it [01:50,  1.09s/it]epoch 1, disc_loss 0.4261084198951721, gen_enc_loss 3.3422980308532715
150it [02:45,  1.10s/it]epoch 1, disc_loss 0.3949705958366394, gen_enc_loss 

### Testing

In [ ]:
!python "/content/drive/MyDrive/BigBiGAN-PyTorch-main/train_gan.py" --dataset "MNIST" --data_path "/content/drive/MyDrive/BigBiGAN-PyTorch-main/data/" --train 0 --loss_mode "no_sz"

original images saved
generated images saved
creating encoded test csv file
39it [00:02, 15.14it/s]
saved encoded
encoded test csv saved
creating encoded train csv file
234it [00:16, 14.00it/s]
saved encoded
encoded train csv saved


### FID

FID:  0.7777843195290473

In [ ]:
!python -m pytorch_fid "/content/drive/MyDrive/BigBiGAN-PyTorch-main/data/org/" "/content/drive/MyDrive/BigBiGAN-PyTorch-main/data/MNIST/bigbigan/no_sz/gen/" --dims 192 --device "cuda"

Downloading: "https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/pt_inception-2015-12-05-6726825d.pth
100% 91.2M/91.2M [00:01<00:00, 58.6MB/s]
100% 6/6 [01:07<00:00, 11.23s/it]
100% 6/6 [00:50<00:00,  8.43s/it]
FID:  0.7777843195290473


### Linear Accuracy

linear accuracy 0.12419871794871795

In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier

train_df = pd.read_csv("/content/drive/MyDrive/BigBiGAN-PyTorch-main/data/MNIST/bigbigan/no_sz/encoded/train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/BigBiGAN-PyTorch-main/data/MNIST/bigbigan/no_sz/encoded/test.csv")

train_y, train_X = train_df.loc[:, "0"], train_df.loc[:, "1":"100"]
test_y, test_X = test_df.loc[:, "0"], test_df.loc[:, "1":"100"]

neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(train_X, train_y)
neigh.score(test_X, test_y)

0.12419871794871795

## No S_XZ

This mode is extra and the question does not ask to calculate statistics for this mode. in this mode the combination of Sx and Sz are occluded but Sx and Sz alone are present in calculation of loss


### Training

--loss_mode: "no_sxz"

epoch 9, disc_loss 1.0040082931518555, gen_enc_loss 0.7531461715698242

In [ ]:
!python "/content/drive/MyDrive/BigBiGAN-PyTorch-main/train_gan.py" --dataset "MNIST" --data_path "/content/drive/MyDrive/BigBiGAN-PyTorch-main/data/" --train 1 --loss_mode "no_sxz"

no_sxz
2021-05-14 09:07:20.028347: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
0it [00:00, ?it/s]epoch 0, disc_loss 2.1846747398376465, gen_enc_loss 1.4773368835449219
50it [00:46,  1.09it/s]epoch 0, disc_loss 0.820879340171814, gen_enc_loss 2.2447586059570312
100it [01:32,  1.07it/s]epoch 0, disc_loss 0.6836590766906738, gen_enc_loss 2.638072967529297
150it [02:19,  1.07it/s]epoch 0, disc_loss 0.7129312753677368, gen_enc_loss 2.3289573192596436
200it [03:06,  1.05it/s]epoch 0, disc_loss 0.7315065264701843, gen_enc_loss 2.9692811965942383
234it [03:38,  1.07it/s]
0it [00:00, ?it/s]epoch 1, disc_loss 0.6966474056243896, gen_enc_loss 2.1858129501342773
50it [00:53,  1.07s/it]epoch 1, disc_loss 0.712207019329071, gen_enc_loss 2.0193371772766113
100it [01:47,  1.09s/it]epoch 1, disc_loss 0.7861006259918213, gen_enc_loss 1.4559266567230225
150it [02:42,  1.10s/it]epoch 1, disc_loss 0.8251285552978516, gen_enc_loss 1.4

### Testing

In [ ]:
!python "/content/drive/MyDrive/BigBiGAN-PyTorch-main/train_gan.py" --dataset "MNIST" --data_path "/content/drive/MyDrive/BigBiGAN-PyTorch-main/data/" --train 0 --loss_mode "no_sxz"

original images saved
creating generated image directory (save one batch
0it [00:00, ?it/s]
  0% 0/256 [00:00<?, ?it/s]
  1% 2/256 [00:00<00:16, 15.57it/s]
  2% 4/256 [00:00<00:18, 13.85it/s]
  2% 5/256 [00:00<00:21, 11.59it/s]
  2% 6/256 [00:00<00:26,  9.58it/s]
  3% 7/256 [00:00<00:29,  8.43it/s]
  3% 8/256 [00:00<00:32,  7.59it/s]
  4% 9/256 [00:01<00:36,  6.75it/s]
  4% 10/256 [00:01<00:40,  6.08it/s]
  4% 11/256 [00:01<00:44,  5.48it/s]
  5% 12/256 [00:01<00:48,  4.99it/s]
  5% 13/256 [00:01<00:52,  4.63it/s]
  5% 14/256 [00:02<00:56,  4.25it/s]
  6% 15/256 [00:02<01:00,  3.97it/s]
  6% 16/256 [00:02<01:04,  3.71it/s]
  7% 17/256 [00:03<01:08,  3.49it/s]
  7% 18/256 [00:03<01:12,  3.30it/s]
  7% 19/256 [00:03<01:16,  3.12it/s]
  8% 20/256 [00:04<01:20,  2.94it/s]
  8% 21/256 [00:04<01:23,  2.80it/s]
  9% 22/256 [00:05<01:27,  2.66it/s]
  9% 23/256 [00:05<01:31,  2.55it/s]
  9% 24/256 [00:05<01:35,  2.44it/s]
 10% 25/256 [00:06<01:39,  2.32it/s]
 10% 26/256 [00:07<01:52,  2.04it/s]

### FID

FID:  5.818132081007027

In [ ]:
!python -m pytorch_fid "/content/drive/MyDrive/BigBiGAN-PyTorch-main/data/org/" "/content/drive/MyDrive/BigBiGAN-PyTorch-main/data/MNIST/bigbigan/no_sxz/gen/" --dims 192 --device "cuda"

Downloading: "https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/pt_inception-2015-12-05-6726825d.pth
100% 91.2M/91.2M [00:00<00:00, 113MB/s]
100% 6/6 [00:28<00:00,  4.79s/it]
100% 6/6 [00:01<00:00,  3.80it/s]
FID:  5.818132081007027


### Linear Accuracy

0.10717147435897435

In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier

train_df = pd.read_csv("/content/drive/MyDrive/BigBiGAN-PyTorch-main/data/MNIST/bigbigan/no_sxz/encoded/train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/BigBiGAN-PyTorch-main/data/MNIST/bigbigan/no_sxz/encoded/test.csv")

train_y, train_X = train_df.loc[:, "0"], train_df.loc[:, "1":"100"]
test_y, test_X = test_df.loc[:, "0"], test_df.loc[:, "1":"100"]

neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(train_X, train_y)
neigh.score(test_X, test_y)

0.10717147435897435

# InfoGAN

in the following codes, a new mode for model training is added: infogan

major changes in the code are in train_gan.py, pipeline.py, architecture.py, model/generator.py, model/discriminator.py and model/losses.py. Whereever there is a change there is a comment section like this: 

\# InfoGan: description of change ###################

my updated code

\###########################################

### Training 

--loss_mode: "info_gan"

if we are in info_gan mode, in pipeline.py a new 10 class uniform random onehot vector is passed to the generator. 

the generator takes a Z vector with length of 100 and a onehot vector (C) with length of 10. It replaces the first 10 values of Z with C and training is done. This information flows through the encoder during training too and the encoder's first 10 values of its output try to reconstruct C.

In the descriminator, a second output is created where it takes the representation of the one but last layer, adds a linear layer with non linear activations and a softmax with 10 variables as output. 

In the loss, if in info_gan mode, the reconstruction loss just like loss_mode: "all" is created and on top of that a new term is added to the loss of both generator and discriminator where both try to minimize it.

this term is the cross entropy of actual C and predicted C in both real and generated images.



In [ ]:
!python "/content/drive/MyDrive/BigBiGAN-PyTorch-main/train_gan.py" --dataset "MNIST" --data_path "/content/drive/MyDrive/BigBiGAN-PyTorch-main/data/" --train 1 --loss_mode "info_gan" --resume 10

Loading data from checkpoint, resuming from epoch: 10
creating model from checkpoint
2021-05-20 11:30:51.752855: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
0it [00:00, ?it/s]epoch 10, disc_loss 12.954505920410156, gen_enc_loss 33.04377365112305
50it [01:16,  1.51s/it]epoch 10, disc_loss 11.627837181091309, gen_enc_loss 9.11083984375
100it [02:31,  1.52s/it]epoch 10, disc_loss 9.46716022491455, gen_enc_loss 9.098752975463867
150it [03:47,  1.51s/it]epoch 10, disc_loss 9.720191955566406, gen_enc_loss 15.017335891723633
200it [05:02,  1.51s/it]epoch 10, disc_loss 9.092279434204102, gen_enc_loss 12.242772102355957
234it [05:54,  1.52s/it]
0it [00:00, ?it/s]epoch 11, disc_loss 9.75343132019043, gen_enc_loss 10.209104537963867
50it [01:16,  1.55s/it]epoch 11, disc_loss 10.225789070129395, gen_enc_loss 5.3207502365112305
100it [02:33,  1.55s/it]epoch 11, disc_loss 10.878839492797852, gen_enc_loss 7.120940208435059
150i

### Testing

Inception Score is ::::::  1.1324967 when 9 epochs

A new function is added to Testing phase in which 100 images are generated and saved in "data/MNIST/infogan/gen_C/" for different values of C like [1,0,0,0,0,0,0,0,0,0] to see if each C corresponds to a visual feature of the generated numbers

In [ ]:
!python "/content/drive/MyDrive/BigBiGAN-PyTorch-main/train_gan.py" --dataset "MNIST" --data_path "/content/drive/MyDrive/BigBiGAN-PyTorch-main/data/" --train 0 --loss_mode "info_gan"

creating C gen directory for InfoGAN
0it [00:00, ?it/s]Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data (

### FID

FID:  5.818133444634549

In [ ]:
!python -m pytorch_fid "/content/drive/MyDrive/BigBiGAN-PyTorch-main/data/org/" "/content/drive/MyDrive/BigBiGAN-PyTorch-main/data/MNIST/bigbigan/no_sxz/gen/" --dims 192 --device "cuda"

Downloading: "https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/pt_inception-2015-12-05-6726825d.pth
100% 91.2M/91.2M [00:03<00:00, 25.4MB/s]
100% 6/6 [01:12<00:00, 12.15s/it]
100% 6/6 [01:11<00:00, 11.84s/it]
FID:  5.818133444634549


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python "/content/drive/MyDrive/BigBiGAN-PyTorch-main/train_gan.py" --dataset "MNIST" --data_path "/content/drive/MyDrive/BigBiGAN-PyTorch-main/data/" --train 1 --loss_mode "info_gan" # --resume 10

creating model from configuration
2021-05-21 15:17:47.695595: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
0it [00:00, ?it/s]epoch 0, disc_loss 15.332979202270508, gen_enc_loss 8.265162467956543
50it [01:54,  2.28s/it]epoch 0, disc_loss 7.582223892211914, gen_enc_loss 15.682806968688965
100it [03:48,  2.29s/it]epoch 0, disc_loss 9.860980987548828, gen_enc_loss 16.33121681213379
150it [05:42,  2.29s/it]epoch 0, disc_loss 9.2186861038208, gen_enc_loss 17.80141830444336
200it [07:37,  2.28s/it]epoch 0, disc_loss 2.5476303100585938, gen_enc_loss 21.89464569091797
234it [08:54,  2.29s/it]
0it [00:00, ?it/s]epoch 1, disc_loss 4.852934837341309, gen_enc_loss 15.166507720947266
50it [02:03,  2.46s/it]epoch 1, disc_loss 6.755472183227539, gen_enc_loss 14.852058410644531
100it [04:06,  2.46s/it]epoch 1, disc_loss 2.7409067153930664, gen_enc_loss 13.804495811462402
150it [06:08,  2.45s/it]epoch 1, disc_loss 4.441460609436035

In [7]:
!python "/content/drive/MyDrive/BigBiGAN-PyTorch-main/train_gan.py" --dataset "MNIST" --data_path "/content/drive/MyDrive/BigBiGAN-PyTorch-main/data/" --train 0 --loss_mode "info_gan"

Traceback (most recent call last):
  File "/content/drive/MyDrive/BigBiGAN-PyTorch-main/train_gan.py", line 106, in <module>
    run_experiments()
  File "/content/drive/MyDrive/BigBiGAN-PyTorch-main/train_gan.py", line 44, in run_experiments
    run_experiment(config)
  File "/content/drive/MyDrive/BigBiGAN-PyTorch-main/train_gan.py", line 74, in run_experiment
    config=config)
  File "/content/drive/MyDrive/BigBiGAN-PyTorch-main/src/pipeline/pipeline.py", line 384, in from_checkpoint
    checkpoint = torch.load(checkpoint_path)
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 585, in load
    with _open_zipfile_reader(opened_file) as opened_zipfile:
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 242, in __init__
    super(_open_zipfile_reader, self).__init__(torch._C.PyTorchFileReader(name_or_buffer))
RuntimeError: [enforce fail at inline_container.cc:145] . PytorchStreamReader failed reading zip archive: failed finding cent

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/BigBiGAN-PyTorch-main/data/MNIST/bigbigan/all/encoded/train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/BigBiGAN-PyTorch-main/data/MNIST/bigbigan/all/encoded/test.csv")

train_y, train_X = train_df.loc[:, "0"], train_df.loc[:, "1":"100"]
test_y, test_X = test_df.loc[:, "0"], test_df.loc[:, "1":"100"]

neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(train_X, train_y)
neigh.score(test_X, test_y)

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/BigBiGAN-PyTorch-main/data/MNIST/bigbigan/all/encoded/train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/BigBiGAN-PyTorch-main/data/MNIST/bigbigan/all/encoded/test.csv")

# Jobs are labels
jobs = train_df.loc[:, "0"]
# workers are predicted C in infoGAN
workers = train_df.loc[:, "1":"10"]
